# Downloading Meade speeches

In [44]:
import re
import bs4
import queue
import json
import sys
import csv
import urllib
import util
import os.path
import datetime
from os.path import join as pjoin
import time
import zipfile
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [16]:
website = "https://www.meade18.com/categorias/discursos/"

In [17]:
def build_soup(website):
    '''
    Takes a website and creates a soup.

    Inputs:
        website: a url

    Outputs:
        Soup object.
    '''

    sitio = util.get_request(website)
    siitio = sitio.text.encode('UTF-16')
    soup = bs4.BeautifulSoup(siitio, "html5lib")

    return soup

In [18]:
#Speech extractor
def extractor(url):
    speech = ""
    soup = build_soup(url)
    for p in soup.find_all('p'):
        speech += p.getText()
    return speech

In [69]:
def str_manager(speech):
    #List of stopwords and meaningless words to remove
    words_to_remove = ['josé', 'antonio', 'meade', 'kuribreña', 'voces', 'coro']
    stop_w = stopwords.words('spanish')
    w_to_remove = words_to_remove + stop_w
    
    #With Meade there are a bunch of periods stuck to the following word.
    speech = speech.replace('.', '. ')
    
    #Tokenize and delete punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(speech.lower())
    
    filtered_words = [word for word in tokens if word not in w_to_remove]
    processed_spch = ' '.join(filtered_words)

    return processed_spch

In [70]:
#builds list of tuples (date,speech)
def soup_to_links(soup, link_list):


    for art in soup.find_all('article'):
        if art.attrs['role'] == "article":
            link = art.find('a')
            url = link.get('href')
            
            speech_raw = extractor(url)
            speech = str_manager(speech_raw)
            
            time = art.find('time')
            date = time.get('datetime')[0:10]
        
            tup = [date, speech]
            link_list.append(tup)

    return(link_list)

In [71]:
#all the links with speeches
all_sites = [website] * 18
for i in range(1,18):
    num = i+1
    plug = str(num)
    all_sites[i] = all_sites[i]+"page/" + plug +"/"

In [72]:
#Putting all together
data = []
for site in all_sites:
    soup = build_soup(site)
    links = soup_to_links(soup, data)

In [75]:
#Building and merging dataframes
meade_df = pd.DataFrame(data, columns = ['date', 'speech']) 
meade_df.to_csv('meade.csv',  encoding='UTF-16', index=False)

In [74]:
meade_df

,date,speech
0,2018-05-27,buenas tardes buenas tardes carmelitas buenas ...
1,2018-05-27,muchas gracias muchas gracias recibirme campec...
2,2018-05-26,buenos días chalco cómo echan ganas días hace ...
3,2018-05-25,buenas tardes fresnillo cómo entusiasmo fresni...
4,2018-05-24,pregunta da gusto saludarte da gusto saludarte...
5,2018-05-24,agradezco aquí agradezco grijalbo presentación...
6,2018-05-23,buenas tardes buenas tardes tlaxcala cómo ánim...
7,2018-05-23,muchas gracias espacio muchas gracias invitaci...
8,2018-05-23,buenos días muchas gracias acompañarme muchas ...
9,2018-05-22,buenas tardes buenas tardes torreón buenas tar...
